In [ ]:
import numpy as np
import pandas as pd
import os
import sys

from statsmodels.tsa.api import STLForecast
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.datasets import macrodata
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.statespace.exponential_smoothing import ExponentialSmoothing
import xgboost as xgb
from statsmodels.tsa.api import SimpleExpSmoothing
from tbats import TBATS, BATS

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_rows', None)

df = pd.read_csv('entities-2023-09-06_10 17 37.csv', usecols=['start', 'mean'])
# df = pd.read_csv('entities-2023-09-02_17 33 41.csv', usecols=['start', 'mean'])
df['start'] = pd.to_datetime(df['start'], format='%Y-%m-%d %H:%M:%S')
df = df.set_index('start')
df = df.asfreq(freq='H').resample('D').sum()
df = df[['mean']]#.head(9*7)

df['dayofweek'] = df.index.dayofweek
df['d1'] = df['mean'].shift(1)
df['d2'] = df['mean'].shift(2)
df['d3'] = df['mean'].shift(3)
df['d7'] = df['mean'].shift(7)
df['d8'] = df['mean'].shift(8)

exogs = ['dayofweek', 'd1', 'd2', 'd3', 'd7']

train_period = 7 * 6
for x in range(0, len(df) - train_period):
# for x in range(1):
    train_start = x
    train_end = x + train_period
    # test_start = train_end
    # test_end = test_start + test_period
    predict_start = train_end
    predict_end = predict_start + 1
    predict_start_date = df.iloc[predict_start].name
    print(f"Training from {train_start} - {train_end - 1}: {df.iloc[train_start].name} - {df.iloc[train_end - 1].name}")

    model = STLForecast(
        df.iloc[train_start:train_end]["mean"],
        SARIMAX,
        model_kwargs={
            "order": (2, 1, 1),
            "enforce_invertibility": False,
            "enforce_stationarity": False,
        },
    )
    results = model.fit(fit_kwargs={"disp": False, "warn_convergence": False})
    # results.model_result.plot_diagnostics()
    model_prediction = results.get_prediction(
        start=df.iloc[train_end].name,
        end=df.iloc[train_end].name
    )
    predicted_mean: pd.DataFrame = model_prediction.predicted_mean.rename("predicted").to_frame()
    df = df.combine_first(predicted_mean)

    X_train = df[train_start:train_end][exogs]
    y_train = df[train_start:train_end]['mean']

    # X_test = df[train_end:train_end][exogs]
    # y_test = df[train_end:train_end]['mean']
    
    X_predict = df[predict_start:predict_end][exogs]

    xgmodel = xgb.XGBRegressor(verbosity=0, reg_lambda=0.2, max_depth=0, booster="gbtree")
    #, eval_set=[(X_train, y_train), (X_test, y_test)]
    xgmodel.fit(X_train, y_train, verbose=False)

    # xgb.plot_importance(xgmodel, height=0.9)

    prediction = xgmodel.predict(X_predict)
    forecasted_df = pd.DataFrame(prediction, columns=['predicted_xg'], index=pd.date_range(predict_start_date, predict_start_date, freq='D'))
    df = df.combine_first(forecasted_df)

    ses = SimpleExpSmoothing(df.iloc[train_start:train_end]["mean"])
    sesmodel = ses.fit(smoothing_level = 0.5, optimized = False)
    sesforecast = sesmodel.forecast(1)
    df = df.combine_first(sesforecast.rename("predicted_ses").to_frame())

    estimator = TBATS(use_box_cox=True, use_trend=True, use_damped_trend=True, seasonal_periods=[7],show_warnings=False)
    fitted_model = estimator.fit(df.iloc[train_start:train_end]['mean'])
    print(fitted_model.summary())
    y_forecasted, _ = fitted_model.forecast(steps=1, confidence_level=0.5)
    tbatsforecast = pd.DataFrame(y_forecasted, columns=['predicted_tbats'], index=pd.date_range(predict_start_date, predict_start_date, freq='D'))
    df = df.combine_first(tbatsforecast)

# print(df)



In [ ]:
# train_period += 4
stl = """{"predicted":{"1689811200000":6.1024130505,"1689897600000":5.2882367582,"1689984000000":7.1046379757,"1690070400000":7.5558324369,"1690156800000":7.4207686308,"1690243200000":7.1760769877,"1690329600000":5.8848356135,"1690416000000":7.0620552752,"1690502400000":5.3280016617,"1690588800000":8.6262117886,"1690675200000":8.2011682559,"1690761600000":9.6541242131,"1690848000000":6.395082486,"1690934400000":6.2387428456,"1691020800000":6.0210112227,"1691107200000":5.9131599036,"1691193600000":6.8258694141,"1691280000000":6.067949583,"1691366400000":5.2549148829,"1691452800000":6.3373241805,"1691539200000":5.5160181919,"1691625600000":7.5489618675,"1691712000000":6.1272265017,"1691798400000":7.850609946,"1691884800000":8.1679921251,"1691971200000":8.8346228733,"1692057600000":7.1282213566,"1692144000000":6.6773378019,"1692230400000":6.6954823189,"1692316800000":7.0140371574,"1692403200000":7.5789723455,"1692489600000":6.9975712542,"1692576000000":7.1835653678,"1692662400000":6.4692300345,"1692748800000":6.2646061006,"1692835200000":6.5575620771,"1692921600000":7.4596137307,"1693008000000":6.8102031764,"1693094400000":6.0367357611,"1693180800000":4.9065749756,"1693267200000":4.1936456354,"1693353600000":4.334267945,"1693440000000":4.7481844442,"1693526400000":4.5254548537,"1693612800000":4.4046905524,"1693699200000":5.1102698367,"1693785600000":10.1143400729,"1693872000000":11.3499860392}}"""
stl = pd.read_json(stl).rename(columns={"predicted": "predicted_fullstl"}).asfreq('D')
df = stl.combine_first(df)

pyplot.figure()
fig, ax = pyplot.subplots(figsize=(15, 5))
df[train_period:]['mean'].plot(ax=ax)
df[train_period:]['predicted'].plot(ax=ax, alpha=0.5)
df[train_period:]['predicted_xg'].plot(ax=ax, alpha=0.5)
df[train_period:]['predicted_ses'].plot(ax=ax, alpha=0.5)
df[train_period:]['predicted_tbats'].plot(ax=ax, alpha=0.5)
df[train_period:]['predicted_fullstl'].plot(ax=ax, alpha=0.5)
df['predicted_mean'] = df[['predicted', 'predicted_xg', 'predicted_ses', 'predicted_tbats', 'predicted_fullstl']].mean(axis=1)
df[train_period:]['predicted_mean'].plot(ax=ax)
# (df[train_period-48:]['mean'] - df[train_period-48:]['predicted']).plot(ax=ax)
# ax.fill_between(df.index, df['predicted_lower'], df['predicted_upper'], color='k', alpha=0.1);  
pyplot.legend()
pyplot.show()

# daily_sums = df[train_period:].resample('D').sum()
df['dow'] = df.index.day_name()
print(df[train_period:][['mean', 'dow', 'predicted', 'predicted_xg', 'predicted_tbats']])
print("Predicted arima")
print(mean_absolute_error(df[train_period:]['mean'], df[train_period:]['predicted']))
print(mean_squared_error(df[train_period:]['mean'], df[train_period:]['predicted']))
print("Predicted XG")
print(mean_absolute_error(df[train_period:]['mean'], df[train_period:]['predicted_xg']))
print(mean_squared_error(df[train_period:]['mean'], df[train_period:]['predicted_xg']))
print("Predicted SES")
print(mean_absolute_error(df[train_period:]['mean'], df[train_period:]['predicted_ses']))
print(mean_squared_error(df[train_period:]['mean'], df[train_period:]['predicted_ses']))
print("Predicted tbats")
print(mean_absolute_error(df[train_period:]['mean'], df[train_period:]['predicted_tbats']))
print(mean_squared_error(df[train_period:]['mean'], df[train_period:]['predicted_tbats']))
print("Predicted Full STL")
print(mean_absolute_error(df[train_period:]['mean'], df[train_period:]['predicted_fullstl']))
print(mean_squared_error(df[train_period:]['mean'], df[train_period:]['predicted_fullstl']))